In [13]:
from sqlalchemy import create_engine
import sqlalchemy
import mysql.connector as mysql
import pandas as pd
from sqlalchemy import text

In [3]:
# Parámetros de conexión
user     = 'root'
password = ''
#db = ''
sqlServer = 'localhost' # debería ser IP o DNS (Domain Name System) del servidor de la base de datos

# Creamos string de conexión
connStr = "mysql+mysqlconnector://" + user + ":" + password + "@" + sqlServer # +  "/"  + db

# print de string de conexión
print (connStr)

# Conectamos con el servidor
conn = create_engine(connStr).connect() 

mysql+mysqlconnector://root:@localhost


In [16]:
# Ruta del archivo CSV
ruta = "/Users/danielalejandroalvarez/Documents/Master Nuclio Python/Entregable FInal/compas-scores-raw.csv"

# Leer CSV
df_raw = pd.read_csv(ruta)

# Cargar a la base de datos como tabla 'compas_raw'
df_raw.to_sql("compas_raw", engine, if_exists="replace", index=False)


60843

**PASO 1: Filtrar solo evaluaciones de reincidencia (Risk of Recidivism)**

In [17]:
# Crear un DataFrame solo con las evaluaciones de reincidencia
df_reincidencia = pd.read_sql("""
SELECT *
FROM compas_raw
WHERE "DisplayText" = 'Risk of Recidivism';
""", engine)

# Mostrar las primeras filas para confirmar
df_reincidencia.head()

,Person_ID,AssessmentID,Case_ID,Agency_Text,LastName,FirstName,MiddleName,Sex_Code_Text,Ethnic_Code_Text,DateOfBirth,...,RecSupervisionLevel,RecSupervisionLevelText,Scale_ID,DisplayText,RawScore,DecileScore,ScoreText,AssessmentType,IsCompleted,IsDeleted
0,50844,57167,51950,PRETRIAL,Fisher,Kevin,None,Male,Caucasian,12/05/92,...,1,Low,8,Risk of Recidivism,-1.06,2,Low,New,1,0
1,50848,57174,51956,PRETRIAL,KENDALL,KEVIN,None,Male,Caucasian,09/16/84,...,1,Low,8,Risk of Recidivism,-1.50,1,Low,New,1,0
2,50855,57181,51963,PRETRIAL,DAYES,DANIEL,None,Male,African-American,08/25/94,...,4,High,8,Risk of Recidivism,0.18,8,High,New,1,0
3,50850,57176,51958,PRETRIAL,Debe,Mikerlie,George,Female,African-American,10/09/94,...,2,Medium,8,Risk of Recidivism,-0.53,6,Medium,New,1,0
4,50839,57162,51945,PRETRIAL,McLaurin,Stephanie,Nicole,Female,African-American,06/29/85,...,1,Low,8,Risk of Recidivism,-1.55,2,Low,New,1,0


### 🧠 Insight - Filtrado por Evaluaciones de Reincidencia

Se ha filtrado la tabla original para trabajar exclusivamente con las evaluaciones de tipo "Risk of Recidivism". Estas son las más relevantes para el análisis de sesgos en COMPAS, ya que corresponden al score utilizado para anticipar reincidencia, influenciando decisiones judiciales como libertad condicional o sentencias preventivas.


**Distribución por Género (Sex_Code_Text)**

In [18]:
pd.read_sql("""
SELECT "Sex_Code_Text" AS genero, COUNT(*) AS cantidad
FROM compas_raw
WHERE "DisplayText" = 'Risk of Recidivism'
GROUP BY "Sex_Code_Text"
ORDER BY cantidad DESC;
""", engine)


,genero,cantidad
0,Male,15838
1,Female,4443


### 🧠 Insight - Distribución de Género

El análisis muestra que la gran mayoría de evaluaciones de reincidencia corresponden a hombres. Esto coincide con tendencias conocidas del sistema penal, pero también implica que cualquier modelo entrenado con estos datos podría estar sesgado hacia patrones predominantes en el género masculino. Es importante considerar este desbalance al interpretar promedios de score y evaluar sesgos potenciales.


**Distribución por Raza (Ethnic_Code_Text)**

In [19]:
pd.read_sql("""
SELECT "Ethnic_Code_Text" AS raza, COUNT(*) AS cantidad
FROM compas_raw
WHERE "DisplayText" = 'Risk of Recidivism'
GROUP BY "Ethnic_Code_Text"
ORDER BY cantidad DESC;
""", engine)


,raza,cantidad
0,African-American,9006
1,Caucasian,7261
2,Hispanic,2914
3,Other,864
4,Asian,108
5,Native American,73
6,Arabic,25
7,African-Am,17
8,Oriental,13


### 🧠 Insight - Distribución por Raza

Los grupos racialmente más representados en las evaluaciones de reincidencia son African-American y Caucasian. Este desbalance demográfico es importante para el análisis porque puede afectar la distribución de los scores, y también puede amplificar cualquier sesgo existente si un grupo aparece mucho más que otros en los datos de entrenamiento y evaluación del sistema COMPAS.


**Promedio del Puntaje COMPAS por Raza**

In [21]:
pd.read_sql("""
SELECT 
  CASE 
    WHEN "Ethnic_Code_Text" = 'African-Am' THEN 'African-American'
    ELSE "Ethnic_Code_Text"
  END AS raza,
  ROUND(AVG("DecileScore"), 2) AS promedio_score
FROM compas_raw
WHERE "DisplayText" = 'Risk of Recidivism'
GROUP BY raza
ORDER BY promedio_score DESC;
""", engine)



,raza,promedio_score
0,African-American,5.32
1,Native American,5.29
2,Arabic,3.84
3,Caucasian,3.59
4,Hispanic,3.31
5,Oriental,3.23
6,Other,2.87
7,Asian,2.57


### 🧠 Insight - Promedio del Score por Raza (agrupado)

Al unificar las etiquetas "African-Am" y "African-American", se observa que este grupo tiene uno de los promedios de score más altos asignados por COMPAS. Esta tendencia sugiere que, incluso sin observar la reincidencia real, el sistema tiende a asignarles un mayor riesgo predicho. Este comportamiento es clave para justificar el análisis de sesgo racial en modelos automatizados.


**Promedio del Puntaje COMPAS por Género**

In [22]:
pd.read_sql("""
SELECT "Sex_Code_Text" AS genero,
       ROUND(AVG("DecileScore"), 2) AS promedio_score
FROM compas_raw
WHERE "DisplayText" = 'Risk of Recidivism'
GROUP BY "Sex_Code_Text"
ORDER BY promedio_score DESC;
""", engine)


,genero,promedio_score
0,Male,4.33
1,Female,4.14


### 🧠 Insight - Promedio del Score por Género

Los resultados muestran que, en promedio, los hombres reciben puntuaciones más altas que las mujeres en las evaluaciones de reincidencia de COMPAS. Esta diferencia puede deberse a patrones históricos en los datos de entrada, pero también puede indicar un posible sesgo estructural si no está completamente justificada por diferencias reales en reincidencia o antecedentes.


In [25]:
query = """
SELECT "Sex_Code_Text" AS genero,
       "ScoreText" AS nivel_riesgo,
       COUNT(*) AS cantidad
FROM compas_raw
WHERE "DisplayText" = 'Risk of Recidivism'
GROUP BY genero, nivel_riesgo
"""

# Ejecutar consulta y pivotear
df_score_genero = pd.read_sql(query, engine).pivot(index='genero', columns='nivel_riesgo', values='cantidad')

# Reordenar columnas manualmente
orden = [None, 'Low', 'Medium', 'High']  # None representa los valores nulos
df_score_genero = df_score_genero[[col for col in orden if col in df_score_genero.columns]].fillna(0)

df_score_genero



nivel_riesgo,Low,Medium,High
genero,,,
Female,2615,1211,610
Male,9004,3776,3029


### 🧠 Insight - Distribución de ScoreText por Género (ordenado por nivel de riesgo)

Al organizar los datos por nivel de riesgo (`ScoreText`), se observa que las mujeres tienen una mayor concentración en la categoría "Low", mientras que los hombres están más distribuidos en las categorías "Medium" y "High". Además, si existen valores nulos en `ScoreText`, suelen estar asociados a una proporción muy baja y podrían indicar evaluaciones incompletas o errores de carga. Esta diferencia refuerza la necesidad de analizar si el sistema COMPAS asocia sistemáticamente mayor riesgo al género masculino.


In [26]:
# Conectar a tu base de datos
engine = create_engine("sqlite:///compas.db")

# Consulta SQL para ScoreText por raza (agrupando African-Am)
query = """
SELECT 
  CASE 
    WHEN "Ethnic_Code_Text" = 'African-Am' THEN 'African-American'
    ELSE "Ethnic_Code_Text"
  END AS raza,
  "ScoreText" AS nivel_riesgo,
  COUNT(*) AS cantidad
FROM compas_raw
WHERE "DisplayText" = 'Risk of Recidivism'
GROUP BY raza, nivel_riesgo
"""

# Ejecutar y pivotear
df_score_raza = pd.read_sql(query, engine).pivot(index='raza', columns='nivel_riesgo', values='cantidad')

# Reordenar columnas
orden = [None, 'Low', 'Medium', 'High']
df_score_raza = df_score_raza[[col for col in orden if col in df_score_raza.columns]].fillna(0)

df_score_raza

nivel_riesgo,Low,Medium,High
raza,,,
African-American,3766.0,2710.0,2534.0
Arabic,16.0,6.0,3.0
Asian,88.0,14.0,6.0
Caucasian,4902.0,1559.0,786.0
Hispanic,2125.0,541.0,241.0
Native American,33.0,21.0,19.0
Oriental,10.0,2.0,1.0
Other,679.0,134.0,49.0


### 🧠 Insight - Distribución de ScoreText por Raza

La tabla muestra la cantidad de evaluaciones de reincidencia realizadas por COMPAS, clasificadas por grupo racial y nivel de riesgo (Low, Medium, High). Cada número representa el total de personas en ese grupo que fueron asignadas a una de esas categorías.

Se observa que ciertos grupos, como African-American, tienen una mayor proporción de evaluaciones clasificadas como "Medium" o "High", en comparación con otros grupos como Caucasian o Asian. Esta diferencia en la distribución de los niveles de riesgo puede ser un indicio de sesgo algorítmico si no está justificada por diferencias reales en reincidencia.

La presentación en formato tabla permite visualizar de forma clara y directa si hay una tendencia del sistema a clasificar con mayor severidad a ciertos grupos raciales.


**Porcentaje de ScoreText por grupo racial**

In [27]:
# Recalcular la tabla original para obtener conteos por raza y nivel de riesgo
query = """
SELECT 
  CASE 
    WHEN "Ethnic_Code_Text" = 'African-Am' THEN 'African-American'
    ELSE "Ethnic_Code_Text"
  END AS raza,
  "ScoreText" AS nivel_riesgo,
  COUNT(*) AS cantidad
FROM compas_raw
WHERE "DisplayText" = 'Risk of Recidivism'
GROUP BY raza, nivel_riesgo
"""

df_conteo = pd.read_sql(query, engine)

# Calcular total por raza
totales_raza = df_conteo.groupby('raza')['cantidad'].sum().reset_index(name='total')

# Unir para calcular porcentajes
df_porcentaje = df_conteo.merge(totales_raza, on='raza')
df_porcentaje['porcentaje'] = round((df_porcentaje['cantidad'] / df_porcentaje['total']) * 100, 2)

# Pivotear para ver porcentaje por nivel de riesgo
df_pivot = df_porcentaje.pivot(index='raza', columns='nivel_riesgo', values='porcentaje').fillna(0)

# Ordenar columnas si existen
orden = [None, 'Low', 'Medium', 'High']
df_pivot = df_pivot[[col for col in orden if col in df_pivot.columns]]

df_pivot

nivel_riesgo,Low,Medium,High
raza,,,
African-American,41.74,30.03,28.08
Arabic,64.00,24.00,12.00
Asian,81.48,12.96,5.56
Caucasian,67.51,21.47,10.82
Hispanic,72.92,18.57,8.27
Native American,45.21,28.77,26.03
Oriental,76.92,15.38,7.69
Other,78.59,15.51,5.67


### 🧠 Insight - Porcentaje de ScoreText por Raza

Esta tabla muestra la distribución porcentual del nivel de riesgo asignado (`ScoreText`) dentro de cada grupo racial. Los resultados reflejan que los grupos African-American y Native American reciben una mayor proporción de evaluaciones clasificadas como “Medium” y “High” en comparación con otros grupos como Caucasian o Asian, que se concentran más en “Low”.

Este análisis proporcional permite detectar posibles sesgos de gravedad, más allá del número absoluto de evaluaciones, revelando cómo el sistema COMPAS tiende a clasificar de forma más severa a ciertos grupos raciales.


**Porcentaje de ScoreText por Género**

In [28]:
# Repetir la consulta de porcentaje de ScoreText por género
query = """
SELECT 
  "Sex_Code_Text" AS genero,
  "ScoreText" AS nivel_riesgo,
  COUNT(*) AS cantidad
FROM compas_raw
WHERE "DisplayText" = 'Risk of Recidivism'
GROUP BY genero, nivel_riesgo
"""

# Ejecutar y procesar
df_conteo_genero = pd.read_sql(query, engine)
totales_genero = df_conteo_genero.groupby('genero')['cantidad'].sum().reset_index(name='total')
df_porcentaje_genero = df_conteo_genero.merge(totales_genero, on='genero')
df_porcentaje_genero['porcentaje'] = round((df_porcentaje_genero['cantidad'] / df_porcentaje_genero['total']) * 100, 2)

# Pivot
df_pivot_genero = df_porcentaje_genero.pivot(index='genero', columns='nivel_riesgo', values='porcentaje').fillna(0)
orden = [None, 'Low', 'Medium', 'High']
df_pivot_genero = df_pivot_genero[[col for col in orden if col in df_pivot_genero.columns]]

df_pivot_genero

nivel_riesgo,Low,Medium,High
genero,,,
Female,58.86,27.26,13.73
Male,56.85,23.84,19.12


### 🧠 Insight - Porcentaje de ScoreText por Género

La distribución proporcional del nivel de riesgo asignado (`ScoreText`) por género muestra una clara diferencia: las mujeres tienen mayor presencia en la categoría “Low”, mientras que los hombres predominan en “Medium” y “High”. Esta brecha puede deberse a diferencias históricas en antecedentes o perfiles delictivos, pero también plantea preguntas sobre posibles sesgos de género en el sistema COMPAS.


**Distribución de AssessmentType por Género**

In [30]:
query = """
SELECT 
  "Sex_Code_Text" AS genero,
  "AssessmentType",
  COUNT(*) AS cantidad
FROM compas_raw
WHERE "DisplayText" = 'Risk of Recidivism'
GROUP BY genero, "AssessmentType"
ORDER BY genero, "AssessmentType";
"""

# Ejecutar la query y mostrar resultados
pd.read_sql(query, engine)



,genero,AssessmentType,cantidad
0,Female,Copy,239
1,Female,New,4204
2,Male,Copy,1329
3,Male,New,14509


In [31]:
query = """
SELECT 
  "Sex_Code_Text" AS genero,
  "AssessmentType",
  COUNT(*) AS cantidad
FROM compas_raw
WHERE "DisplayText" = 'Risk of Recidivism'
  AND "AssessmentType" IS NOT NULL
GROUP BY genero, "AssessmentType";
"""
df_assess = pd.read_sql(query, engine)

# Calcular total por género
totales = df_assess.groupby('genero')['cantidad'].sum().reset_index(name='total')
df_assess = df_assess.merge(totales, on='genero')
df_assess['porcentaje'] = round((df_assess['cantidad'] / df_assess['total']) * 100, 2)

# Pivot para mejor visualización
df_assess_pivot = df_assess.pivot(index='genero', columns='AssessmentType', values='porcentaje').fillna(0)
df_assess_pivot


AssessmentType,Copy,New
genero,,
Female,5.38,94.62
Male,8.39,91.61


### 🧠 Insight - Porcentaje de Tipos de Evaluación por Género

Al observar la distribución proporcional de los tipos de evaluación (`AssessmentType`) entre hombres y mujeres, se confirma que ambos géneros son evaluados mayoritariamente con el tipo “New”. Sin embargo, los hombres presentan un porcentaje ligeramente mayor de evaluaciones clasificadas como “Copy”, lo que podría implicar la reutilización de scores anteriores. Este patrón debe considerarse, ya que podría introducir variaciones no deseadas en la asignación del riesgo.


## 📘 Resumen del Análisis Exploratorio en SQL – COMPAS

Este EDA se realizó directamente desde SQL sobre el dataset `compas-scores-raw.csv`, cargado en una base de datos SQLite como tabla `compas_raw`. El análisis se enfocó únicamente en las evaluaciones de tipo `Risk of Recidivism`.

---

### ✅ Consultas y análisis realizados:

1. **Filtrado de Evaluaciones de Reincidencia**
   - Se seleccionaron únicamente las entradas con `DisplayText = 'Risk of Recidivism'`.

2. **Distribución por Género y Raza**
   - Se contó cuántas evaluaciones corresponden a cada grupo.
   - Se detectó un desbalance claro, predominando hombres y personas African-American y Caucasian.

3. **Promedio del Puntaje COMPAS (`DecileScore`)**
   - Calculado por raza y por género.
   - Se observó que los hombres y personas African-American tienden a recibir puntajes más altos.

4. **Distribución de ScoreText (Low / Medium / High)**
   - Se mostró en tabla pivoteada por género y por raza.
   - También se calculó el porcentaje de cada nivel de riesgo dentro de cada grupo, revelando diferencias significativas.

5. **Tipo de Evaluación (`AssessmentType`)**
   - Se analizó por género, mostrando que la mayoría de evaluaciones son de tipo "New", aunque los hombres presentan un mayor uso de "Copy".

---

### 🧠 Conclusión

Este análisis inicial revela diferencias notables en cómo el sistema COMPAS clasifica a las personas según su género y raza, incluso antes de evaluar si esas clasificaciones son precisas o no. Estas diferencias justifican un análisis más profundo o el desarrollo de un modelo alternativo menos dependiente de variables sensibles.



**Cargar compas-scores-two-years.csv como tabla compas_two_years**


In [33]:
# Ruta al archivo
ruta = "/Users/danielalejandroalvarez/Documents/Master Nuclio Python/Entregable FInal/compas-scores-two-years.csv"

# Leer CSV
df_two_years = pd.read_csv(ruta)

# Subir a SQL como nueva tabla
df_two_years.to_sql("compas_two_years", engine, if_exists="replace", index=False)

7214

In [34]:
pd.read_sql("SELECT * FROM compas_two_years LIMIT 5;", engine)


,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,6,Medium,2013-01-13,None,None,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,1,Low,2013-03-26,None,None,2,0,1102,0,0


**Clasificar predicciones COMPAS y resultado real**

In [38]:
query = """
SELECT *,
       CASE 
         WHEN "decile_score" >= 5 THEN 1
         ELSE 0
       END AS compas_pred
FROM compas_two_years;
"""
pd.read_sql(query, engine)

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid,compas_pred
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1,0
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1,0
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,Medium,2013-01-13,None,None,1,0,1174,0,0,1
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,Low,2013-03-26,None,None,2,0,1102,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7209,10996,steven butler,steven,butler,2013-11-23,Male,1992-07-17,23,Less than 25,African-American,...,Medium,2013-11-23,2013-11-22,2013-11-24,0,1,860,0,0,1
7210,10997,malcolm simmons,malcolm,simmons,2014-02-01,Male,1993-03-25,23,Less than 25,African-American,...,Medium,2014-02-01,2014-01-31,2014-02-02,0,1,790,0,0,0
7211,10999,winston gregory,winston,gregory,2014-01-14,Male,1958-10-01,57,Greater than 45,Other,...,Low,2014-01-14,2014-01-13,2014-01-14,0,0,808,0,0,0
7212,11000,farrah jean,farrah,jean,2014-03-09,Female,1982-11-17,33,25 - 45,African-American,...,Low,2014-03-09,2014-03-08,2014-03-09,3,0,754,0,0,0


In [37]:
query = """
SELECT *,
  CASE
    WHEN "decile_score" >= 5 AND "two_year_recid" = 1 THEN 'true_positive'
    WHEN "decile_score" >= 5 AND "two_year_recid" = 0 THEN 'false_positive'
    WHEN "decile_score" < 5 AND "two_year_recid" = 0 THEN 'true_negative'
    WHEN "decile_score" < 5 AND "two_year_recid" = 1 THEN 'false_negative'
    ELSE 'error'
  END AS compas_outcome
FROM compas_two_years;
"""
pd.read_sql(query, engine)


,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid,compas_outcome
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0,true_negative
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1,false_negative
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1,false_negative
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,Medium,2013-01-13,None,None,1,0,1174,0,0,false_positive
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,Low,2013-03-26,None,None,2,0,1102,0,0,true_negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7209,10996,steven butler,steven,butler,2013-11-23,Male,1992-07-17,23,Less than 25,African-American,...,Medium,2013-11-23,2013-11-22,2013-11-24,0,1,860,0,0,false_positive
7210,10997,malcolm simmons,malcolm,simmons,2014-02-01,Male,1993-03-25,23,Less than 25,African-American,...,Medium,2014-02-01,2014-01-31,2014-02-02,0,1,790,0,0,true_negative
7211,10999,winston gregory,winston,gregory,2014-01-14,Male,1958-10-01,57,Greater than 45,Other,...,Low,2014-01-14,2014-01-13,2014-01-14,0,0,808,0,0,true_negative
7212,11000,farrah jean,farrah,jean,2014-03-09,Female,1982-11-17,33,25 - 45,African-American,...,Low,2014-03-09,2014-03-08,2014-03-09,3,0,754,0,0,true_negative


**Conteo de errores de predicción (compas_outcome)**

In [39]:
query = """
SELECT 
  CASE
    WHEN "decile_score" >= 5 AND "two_year_recid" = 1 THEN 'true_positive'
    WHEN "decile_score" >= 5 AND "two_year_recid" = 0 THEN 'false_positive'
    WHEN "decile_score" < 5 AND "two_year_recid" = 0 THEN 'true_negative'
    WHEN "decile_score" < 5 AND "two_year_recid" = 1 THEN 'false_negative'
    ELSE 'error'
  END AS compas_outcome,
  COUNT(*) AS cantidad
FROM compas_two_years
GROUP BY compas_outcome;
"""

pd.read_sql(query, engine)


,compas_outcome,cantidad
0,false_negative,1216
1,false_positive,1282
2,true_negative,2681
3,true_positive,2035


### 🧠 Insight - Conteo de Errores de Predicción COMPAS

Esta tabla resume el rendimiento básico del sistema COMPAS comparando sus predicciones (según `decile_score`) con los resultados reales (`two_year_recid`). Los falsos positivos son especialmente importantes: representan personas a las que se les predijo un alto riesgo de reincidencia pero no reincidieron, lo que puede reflejar decisiones judiciales potencialmente injustas. Los falsos negativos también son críticos, ya que suponen una subestimación del riesgo real.


**Errores de predicción por etnia


In [44]:
# Primero verificar qué columnas están disponibles
print("Verificando columnas disponibles...")
test_query = "SELECT * FROM compas_two_years LIMIT 5"
test_df = pd.read_sql(test_query, engine)
print("Columnas disponibles:")
print(test_df.columns.tolist())
print()

# También verificar algunos valores de ejemplo para la columna de etnicidad
print("Buscando columna de etnicidad...")
for col in test_df.columns:
    if 'ethnic' in col.lower() or 'race' in col.lower():
        print(f"Columna encontrada: {col}")
        print(f"Valores únicos: {test_df[col].unique()}")
        print()

# Query corregida (ajustar nombre de columna según lo que encuentres arriba)
query = """
SELECT
    CASE
        WHEN race IN ('African-Am', 'African-American') THEN 'African-American'
        WHEN race LIKE '%Caucasian%' THEN 'Caucasian'
        WHEN race LIKE '%Asian%' THEN 'Asian'
        WHEN race LIKE '%Hispanic%' THEN 'Hispanic'
        WHEN race LIKE '%Native%' THEN 'Native American'
        WHEN race IS NULL OR TRIM(race) = '' THEN 'Desconocido'
        ELSE race
    END AS grupo_etnico,
    CASE
        WHEN decile_score >= 5 AND two_year_recid = 1 THEN 'true_positive'
        WHEN decile_score >= 5 AND two_year_recid = 0 THEN 'false_positive'
        WHEN decile_score < 5 AND two_year_recid = 0 THEN 'true_negative'
        WHEN decile_score < 5 AND two_year_recid = 1 THEN 'false_negative'
        ELSE 'error'
    END AS compas_outcome,
    COUNT(*) AS cantidad
FROM compas_two_years
GROUP BY 1, 2
ORDER BY 1, 2;
"""

Verificando columnas disponibles...
Columnas disponibles:
['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob', 'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score', 'juv_misd_count', 'juv_other_count', 'priors_count', 'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number', 'c_offense_date', 'c_arrest_date', 'c_days_from_compas', 'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number', 'r_charge_degree', 'r_days_from_arrest', 'r_offense_date', 'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid', 'is_violent_recid', 'vr_case_number', 'vr_charge_degree', 'vr_offense_date', 'vr_charge_desc', 'type_of_assessment', 'decile_score.1', 'score_text', 'screening_date', 'v_type_of_assessment', 'v_decile_score', 'v_score_text', 'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1', 'start', 'end', 'event', 'two_year_recid']

Buscando columna de etnicidad...
Columna encontrada: race
Valores únicos: ['Other' 'African-American']



In [45]:
# Primero ver todos los valores únicos en la columna race
print("Verificando todos los valores únicos en 'race':")
race_query = "SELECT DISTINCT race, COUNT(*) as count FROM compas_two_years GROUP BY race ORDER BY count DESC"
race_df = pd.read_sql(race_query, engine)
print(race_df)
print()

# Query corregida con los nombres correctos de columna
query = """
SELECT
    CASE
        WHEN race IN ('African-Am', 'African-American') THEN 'African-American'
        WHEN race = 'Caucasian' THEN 'Caucasian'
        WHEN race = 'Asian' THEN 'Asian'
        WHEN race = 'Hispanic' THEN 'Hispanic'
        WHEN race = 'Native American' THEN 'Native American'
        WHEN race = 'Other' THEN 'Other'
        WHEN race IS NULL OR TRIM(race) = '' THEN 'Desconocido'
        ELSE race
    END AS grupo_etnico,
    CASE
        WHEN decile_score >= 5 AND two_year_recid = 1 THEN 'true_positive'
        WHEN decile_score >= 5 AND two_year_recid = 0 THEN 'false_positive'
        WHEN decile_score < 5 AND two_year_recid = 0 THEN 'true_negative'
        WHEN decile_score < 5 AND two_year_recid = 1 THEN 'false_negative'
        ELSE 'error'
    END AS compas_outcome,
    COUNT(*) AS cantidad
FROM compas_two_years
GROUP BY 1, 2
ORDER BY 1, 2;
"""

# Ejecutar la query
print("Ejecutando query principal:")
df = pd.read_sql(query, engine)
print(df)

Verificando todos los valores únicos en 'race':
               race  count
0  African-American   3696
1         Caucasian   2454
2          Hispanic    637
3             Other    377
4             Asian     32
5   Native American     18

Ejecutando query principal:
        grupo_etnico  compas_outcome  cantidad
0   African-American  false_negative       532
1   African-American  false_positive       805
2   African-American   true_negative       990
3   African-American   true_positive      1369
4              Asian  false_negative         3
5              Asian  false_positive         2
6              Asian   true_negative        21
7              Asian   true_positive         6
8          Caucasian  false_negative       461
9          Caucasian  false_positive       349
10         Caucasian   true_negative      1139
11         Caucasian   true_positive       505
12          Hispanic  false_negative       129
13          Hispanic  false_positive        87
14          Hispanic   true_n

In [46]:
# Paso 1: Ejecutar query original
query = """
SELECT
    CASE
        WHEN race IN ('African-Am', 'African-American') THEN 'African-American'
        WHEN race = 'Caucasian' THEN 'Caucasian'
        WHEN race = 'Asian' THEN 'Asian'
        WHEN race = 'Hispanic' THEN 'Hispanic'
        WHEN race = 'Native American' THEN 'Native American'
        WHEN race = 'Other' THEN 'Other'
        WHEN race IS NULL OR TRIM(race) = '' THEN 'Desconocido'
        ELSE race
    END AS grupo_etnico,
    CASE
        WHEN decile_score >= 5 AND two_year_recid = 1 THEN 'true_positive'
        WHEN decile_score >= 5 AND two_year_recid = 0 THEN 'false_positive'
        WHEN decile_score < 5 AND two_year_recid = 0 THEN 'true_negative'
        WHEN decile_score < 5 AND two_year_recid = 1 THEN 'false_negative'
        ELSE 'error'
    END AS compas_outcome,
    COUNT(*) AS cantidad
FROM compas_two_years
GROUP BY 1, 2
ORDER BY 1, 2;
"""

df_errores = pd.read_sql(query, engine)

# Paso 2: Calcular total por grupo
totales = df_errores.groupby('grupo_etnico')['cantidad'].sum().reset_index(name='total')
df_porcentaje = df_errores.merge(totales, on='grupo_etnico')
df_porcentaje['porcentaje'] = round((df_porcentaje['cantidad'] / df_porcentaje['total']) * 100, 2)

# Paso 3: Tabla final (pivot)
tabla_errores_pct = df_porcentaje.pivot(index='grupo_etnico', columns='compas_outcome', values='porcentaje').fillna(0)
tabla_errores_pct


compas_outcome,false_negative,false_positive,true_negative,true_positive
grupo_etnico,,,,
African-American,14.39,21.78,26.79,37.04
Asian,9.38,6.25,65.62,18.75
Caucasian,18.79,14.22,46.41,20.58
Hispanic,20.25,13.66,49.92,16.17
Native American,5.56,16.67,27.78,50.00
Other,23.87,9.55,55.17,11.41


### 🧠 Insight - Porcentaje de Errores COMPAS por Grupo Étnico

Esta tabla permite visualizar de forma clara y proporcional cómo se distribuyen los errores de predicción dentro de cada grupo étnico. Si ciertos grupos presentan una tasa elevada de falsos positivos o falsos negativos, puede ser un indicio de sesgo algorítmico. Por ejemplo, un alto porcentaje de `false_positive` en un grupo puede significar que COMPAS sobreestima el riesgo en ese grupo específico.


**Porcentaje de Errores COMPAS por Género (tabla)**

In [47]:
# Paso 1: Ejecutar query con clasificación de errores por género
query = """
SELECT
    sex AS genero,
    CASE
        WHEN decile_score >= 5 AND two_year_recid = 1 THEN 'true_positive'
        WHEN decile_score >= 5 AND two_year_recid = 0 THEN 'false_positive'
        WHEN decile_score < 5 AND two_year_recid = 0 THEN 'true_negative'
        WHEN decile_score < 5 AND two_year_recid = 1 THEN 'false_negative'
        ELSE 'error'
    END AS compas_outcome,
    COUNT(*) AS cantidad
FROM compas_two_years
GROUP BY genero, compas_outcome
ORDER BY genero, compas_outcome;
"""

df_errores_genero = pd.read_sql(query, engine)

# Paso 2: Calcular totales por género y porcentajes
totales_gen = df_errores_genero.groupby('genero')['cantidad'].sum().reset_index(name='total')
df_porcentaje_gen = df_errores_genero.merge(totales_gen, on='genero')
df_porcentaje_gen['porcentaje'] = round((df_porcentaje_gen['cantidad'] / df_porcentaje_gen['total']) * 100, 2)

# Paso 3: Pivot para tabla final
tabla_errores_genero = df_porcentaje_gen.pivot(index='genero', columns='compas_outcome', values='porcentaje').fillna(0)
tabla_errores_genero


compas_outcome,false_negative,false_positive,true_negative,true_positive
genero,,,,
Female,13.98,20.65,43.66,21.72
Male,17.55,17.08,35.61,29.76


### 🧠 Insight - Porcentaje de Errores COMPAS por Género

Al analizar la distribución proporcional de los errores de COMPAS por género, se observa si existen diferencias sistemáticas en la forma en que el sistema falla para hombres y mujeres. Si, por ejemplo, los hombres presentan más falsos positivos y las mujeres más falsos negativos, esto puede indicar un sesgo de clasificación basado en género, con consecuencias diferentes para cada grupo.


ModuleNotFoundError: No module named 'plotly'